## Genetic Algorithm for Generating test vectors 
- We try to get optimal vectors for testing NoC router using genetic Algorithm

In [70]:
from numpy.random import randint
from numpy.random import rand
from random import sample
import time
import numpy as np
import random

In [71]:
# Parameters

rows = 4
cols = 4
n_iter = 100 # no of generations
n_bits = rows *cols # no of bits in a chromosome
n_pop = 20 # no of chromosomes in a population
r_cross = 0.9
r_mut = 0.9 #1.0 / float(n_bits)
print("r_mut :", r_mut)

r_mut : 0.9


every chromosome is a 2D array. Why? I dont know
and we send a pop, which is an array of chromosomes

In [72]:
# Fitness function
def fitness(sequence):
    #print(sequence)
    covered_comb = 0
    total_poss_comb = n_bits*n_bits-n_bits
    #compare test particle with soultion
    for i in range(n_bits):
        for j in range(n_bits-1):
        #if (solution[i]== 1):
         #   total_poss_comb = total_poss_comb + 1
            if (sequence[i][j]  == 1):
                covered_comb = covered_comb + 1

    fitness_value = (covered_comb/ total_poss_comb) * 100
    #print(fitness_value)
    return fitness_value


# Selects the best 2 chromosomes
def selection(pop, scores):
    #print(scores)
    selected = []
    for i in range(2):
        l = scores.index(max(scores))
        selected.append(pop[l])
        scores[l] = -1
    return selected

def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    p1 = [item for sublist in p1 for item in sublist]
    p2 = [item for sublist in p2 for item in sublist]
    #p1 = list(p1)
    #p2 = list(p2)
    # check for recombination

    if rand() < r_cross:
        k = randint(0, len(p1))
        #print("Crossover point :", k)

        # interchanging the genes
        # for i in range(k, len(p1)):
        #    p1[i], p2[i] = p2[i], p1[i]
        # p1 = ''.join(p1)
        # p2 = ''.join(p2)
        c1 = np.append(p1[:k], p2[k:])
        c2 = np.append(p2[:k], p1[k:])
        #print(c1)
        #print(c2, "\n\n")
        ch1=c1
        #return [p1, p2]
    else:
        ch1 = p1
    return ch1
# Child is flattened, not in 2d array form


# Mutate the string(flip the value) if cond met
def mutation(bitstring, r_mut):
    for i in range(cols+1):
        if rand() < r_mut:
            a = random.randint(0,len(bitstring)-1)
            #print("Flip point :", a)
            if bitstring[a]:
                bitstring[a] = 0
            else:
                bitstring[a] = 1

    return bitstring

# Remove chromosomes with same fitness. If any children are same, remove it
def redundancy_check(n_bits,pop):
    cov = []
    valid = []
    for i in range(len(pop)):
        t = 0
        for j in range(n_bits):
            for k in range(n_bits-1):
                #print(type(pop[i][j][k]))
                if pop[i][j][k] == 1:
                    t=t+1
        cov.append(t)
    for i in range(len(pop)):
        if cov[i] not in cov[:i]:
            valid.append(pop[i])
    return valid

# If child is same as any chromosome in population
def check_valid(child,pop): # Inefficient. Need to rewrite it
    t1 = fitness(child)
    #print("inside!!!!!!!!!!!!!!!!!11")
    #print(pop)
    for i in range(len(pop)):
        t2=fitness(pop[i])
        if (t1==t2):
            return None
    return child

# genetic algorithm
def genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut,new_pop):
    # initial population
    pop=new_pop
    flag=False  # To check if saturation is reached
    print("Initial Population: ")
    # print(pop)

    # Assume an initial best solution
    best, best_eval = pop[0], fitness(pop[0])

    # Iterate generations
    for gen in range(n_iter):
        print(f"GEN= {gen}")

        # Evaluate scores
        scores = [fitness(c) for c in pop]
        print(f"Scores: {scores}")

        # Check if saturation is reached
        for i in range(len(scores)):
            if (scores[i] == 100):
                flag = True

        # check for new best solution
        for i in range(len(pop)):
            if (scores[i] >best_eval):
                best, best_eval = pop[i], scores[i]
                #print(best, best_eval)
        print(f"New best solution in generation {gen}, fitness: {best_eval}")

        if (flag):
            break


        # select parents for crossing into next gen
        selected = selection(pop, scores)

        # create the next generation
        children = []
        # pop_fitness = [fitness(c) for c in pop]  # Pre-calculate population fitness for valid check

        for i in range(n_pop):
            # get selected parents in pairs
            p1 = selected[0]
            p2 = selected[1]
            # crossover. Return 1 child
            c=crossover(p1, p2, r_cross)

            # Mutation of child
            ch=mutation(c, r_mut)

            # Reshape the child to the original chromosome format
            child = np.array(c).reshape(n_bits, n_bits - 1)
            
            # Check if the child is valid (unique fitness) before adding it
            valid_child = check_valid(child, pop)


            if valid_child is not None:
                children.append(valid_child)
                # store for next generation

        # Redundancy check: Ensure no redundant chromosomes in the population
        # pop = redundancy_check(n_bits, children)
        pop = children
        # If population size falls short after redundancy check, regenerate random children
        # while len(pop) < n_pop:
        #     # Generate a random child
        #     rand_child = np.random.randint(0, 2, size=(n_bits, n_bits - 1))
        #     if check_valid(rand_child, pop) is not None:
        #         pop.append(rand_child)

        # Print updated population after redundancy and valid check
        print(f"Updated population in generation {gen}:")
        # print(pop)
    return [gen,best, best_eval]


In [ ]:
ss = []

for i in range(n_pop):
    particle = np.random.randint(2, size=(n_bits, n_bits-1))
    #print(particle)
    #particle.tolist()
    #print(particle)
    ss.append(particle.tolist())

new_pop=ss
start_time = time.time()
# rows and columns

# perform the genetic algorithm search
gen,best, score = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut,new_pop)
print("GEN= {}".format(gen))
print("BEST= {},Score= {}".format(best,score))
print("---- Execution Time = {} seconds ----".format(np.around((time.time() - start_time),decimals=2)))



lst_s = list(range(0, n_bits))
lst_d = list(range(0, n_bits-1))

combos=[]
f=open("traffic"+str(rows)+"x"+str(cols)+".txt","w")
for i in lst_s:
    for j in lst_d:
        combos.append([i,j])
random.shuffle(combos)

for k in combos:
    j=k[1]
    i=k[0]
    if (j >= i):
        d = j + 1
    else:
        d = j
    if (best[i][j] >= 1):
        lbs = i // cols
        source_l = bin(lbs).replace('0b', '')
        x = source_l[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        source_l = x[::-1]
        ubs = i % cols
        source_u = bin(ubs).replace('0b', '')
        x = source_u[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        source_u = x[::-1]
        source = str(source_l) + str(source_u)
        ######################
        lbd = d // cols
        dest_l = bin(lbd).replace('0b', '')
        x = dest_l[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        dest_l = x[::-1]
        ubd = d % cols
        dest_u = bin(ubd).replace('0b', '')
        x = dest_u[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        dest_u = x[::-1]
        dest = str(dest_l) + str(dest_u)

        rand_ls = np.random.random_integers(0, 1, size=(13))
        rand_ls = str(rand_ls).replace(" ", "")
        rand_ls = str(rand_ls).replace("[", "")
        rand_ls = str(rand_ls).replace("]", "")
        h_f = str(source) + str(dest)
        f.write(h_f)
        f.write("\n")
f.close()


In [ ]:
# Define initial population
def initialize_population(n_pop, n_bits):
    pop = []
    for i in range(n_pop):
        a = np.random.randint(0, 2, size=(n_bits, n_bits - 1))
        pop.append(a)
    return pop

new_pop = initialize_population(n_pop, n_bits)
# print(new_pop)
gen, best_chromosome, best_fitness = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut, new_pop)

print(f"Best chromosome found at generation {gen}:")
print(best_chromosome)
print(f"Fitness of the best chromosome: {best_fitness:.2f}")

In [ ]:
from numpy.random import randint
from numpy.random import rand
from random import sample
import time
import numpy as np
import random

########Parameters
rows = 4
columns = 4
# define the total iterations
n_iter = 100
# bits
n_bits = rows *columns
# define the population size
n_pop = 20
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 0.9#1.0 / float(n_bits)
print("r_mut :", r_mut)
##########

def fitness(sequence):
    #print("SeQQQQQQQQQQ")
    #print(sequence)
    covd = 0
    total_poss_comb = n_bits*n_bits-n_bits
    #compare test particle with soultion
    for i in range(n_bits):
        for j in range(n_bits-1):
        #if (solution[i]== 1):
         #   total_poss_comb = total_poss_comb + 1
            if (sequence[i][j]  == 1):
                covd = covd + 1

    fitness_value = (covd/ total_poss_comb) * 100
    #print(fitness_value)
    return fitness_value


# tournament selection
def selection(pop, scores):
    #print(scores)
    selected = []
    for i in range(2):
        l = scores.index(max(scores))
        selected.append(pop[l])
        scores[l] = -1
    return selected


# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    p1 = [item for sublist in p1 for item in sublist]
    p2 = [item for sublist in p2 for item in sublist]
    #p1 = list(p1)
    #p2 = list(p2)
    # check for recombination

    if rand() < r_cross:
        k = randint(0, len(p1))
        #print("Crossover point :", k)

        # interchanging the genes
        # for i in range(k, len(p1)):
        #    p1[i], p2[i] = p2[i], p1[i]
        # p1 = ''.join(p1)
        # p2 = ''.join(p2)
        c1 = np.append(p1[:k], p2[k:])
        c2 = np.append(p2[:k], p1[k:])
        #print(c1)
        #print(c2, "\n\n")
        ch1=c1
        #return [p1, p2]
    else:
        ch1 = p1
    return ch1

def mutation(bitstring, r_mut):
        # check for a mutation
    for i in range(columns+1):
        if rand() < r_mut:
            a = random.randint(0,len(bitstring)-1)
            #print("Flip point :", a)
            if bitstring[a]:
                bitstring[a] = 0
            else:
                bitstring[a] = 1

    return bitstring
#######(R+NR)
ss = []

for i in range(n_pop):
    particle = np.random.randint(2, size=(n_bits, n_bits-1))
    #print(particle)
    #particle.tolist()
    #print(particle)
    ss.append(particle.tolist())
#print(ss)

#####only NR
def redundancy_check(n_bits,pop):#,pop_size,iter_size):
    cov = []
    valid = []
    #flat_pop = [item for sublist in pop for item in sublist]
    for i in range(len(pop)):
        t = 0
        for j in range(n_bits):
            for k in range(n_bits-1):
                #print(type(pop[i][j][k]))
                if pop[i][j][k] == 1:
                    t=t+1
        cov.append(t)
    for i in range(len(pop)):
        if cov[i] not in cov[:i]:
            valid.append(pop[i])
    return valid

def check_valid(child,pop):
    t1 = fitness(child)
    #print("inside!!!!!!!!!!!!!!!!!11")
    #print(pop)
    for i in range(len(pop)):
        t2=fitness(pop[i])
        if (t1==t2):
            return []
    return child

#new_pop=redundancy_check(n_bits,ss)
new_pop=ss
#print(len(new_pop))
# genetic algorithm
def genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut,new_pop):
    # initial population of random bitstring
    """pop = []
    for i in range(n_pop):
        a = np.random.random_integers(0, 1, size=(n_bits, n_bits))
        sequence = np.tril(a) + np.tril(a, -1).T
        np.fill_diagonal(sequence, 0)
        pop.append(list(sequence.flatten()))
    print("Initial Population: ")
    print(pop)"""
    # keep track of best solution
    pop=new_pop
    #print(pop)
    flag=False
    print("Initial Population: ")
    #print(pop)
    best, best_eval = pop[0], fitness(pop[0])
    #print(best_eval)
    # print(objective(pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        print("GEN= {}".format(gen))
        # evaluate all candidates in the population
        #print(pop)

        scores = [fitness(c) for c in pop]

        print(pop)
        print(scores)
        #print(scores)
        for i in range(len(scores)):
            if (scores[i] == 100):
                flag = True

        # check for new best solution
        for i in range(len(pop)):
            if (scores[i] >best_eval):
                best, best_eval = pop[i], scores[i]
                #print(best, best_eval)
                print("iteration {}, new best f({}) = {}".format(i, pop[i], scores[i]))
                
            # select parents
        selected = selection(pop, scores)
        #print("SELECTED")
        #print(selected)
        # print(selected)
        # create the next generation
        children = []
        if (flag):
            break
        for i in range(0, len(pop)):
            # get selected parents in pairs
            #x = randint(len(selected))
            #y = randint(len(selected))
            #while x == y:
            #    x = randint(len(selected))
             #   y = randint(len(selected))

            p1 = selected[0]
            p2 = selected[1]

            #print(p1,p2)
            # crossover and mutation
            #while True:
            c=crossover(p1, p2, r_cross)

                    # mutation
            #print("AFTER CROSSOVER")
            #print(c)

            ch=mutation(c, r_mut)
                    #####################
                    #now check valid here
            #print("AFTER MUTATION")

            ch=np.array(ch)
            ch = ch.reshape(n_bits, n_bits-1)
                #check=check_valid(ch,pop)

                #if(check!=[]):
                #    print("CHECK")
                #    print(check)
            #print(ch)
            children.append(ch)
                    #print(pop)
                    #break
            #break
                ####################
                # store for next generation


            # replace population
        pop = children
        #print(pop)
        #best_m = np.reshape(best, (n_bits, n_bits))
        #print(best_m)


        if (flag):
            break

        print("BEST= {},Score= {}".format(best,best_eval))
        print("---- Execution Time = {} seconds ----".format(np.around((time.time() - start_time),decimals=2)))
    return [gen,best, best_eval]



start_time = time.time()
# rows and columns

# perform the genetic algorithm search
gen,best, score = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut,new_pop)
print("GEN= {}".format(gen))
print("BEST= {},Score= {}".format(best,score))
print("---- Execution Time = {} seconds ----".format(np.around((time.time() - start_time),decimals=2)))

######TRAFFIC GENERATION
total=n_bits*n_bits-n_bits
payloads=3
#test=np.ones([n_bits,n_bits-1])

"""
test= [[1 ,1, 1, 1 ,1 ,0 ,1, 1, 1, 1,1, 1, 1, 1 ,0, 1],
 [1 ,1 ,1, 1, 1, 1, 1, 0, 1, 1 ,1 ,1, 1, 1, 1, 1],
 [1 ,1, 1, 0 ,1 ,0 ,0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [0 ,1 ,1 ,1 ,1, 1 ,0 ,0 ,0 ,0 ,1 ,1, 0, 1, 1, 1],
 [1 ,1, 0 ,1, 1 ,0 ,0 ,0 ,1 ,1 ,1, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 1 ,0 ,0 ,0 ,1 ,1 ,1 ,0 ,0 ,0],
 [0 ,1 ,1, 1 ,1 ,0 ,0 ,1 ,0 ,0 ,1 ,1 ,0 ,0, 1 ,1],
 [0 ,1 ,1 ,1 ,1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
 [0 ,1, 1 ,0, 1 ,1 ,1 ,1 ,1 ,1 ,0 ,1 ,0 ,1 ,1 ,1],
 [0 ,0, 0, 1, 1 ,1, 1, 1, 1 ,1 ,0 ,1 ,0 ,1 ,1 ,1],
 [0 ,0 ,1 ,0 ,0 ,0 ,1 ,0 ,1 ,0 ,1 ,1 ,0 ,0 ,0 ,1],
 [0 ,1 ,1 ,1 ,1 ,1 ,1, 0, 1, 0 ,1 ,0 ,0, 1, 0 ,1],
 [1 ,0, 0, 1, 0 ,0 ,0, 0,1, 1, 0 ,0 ,0 ,1 ,1 ,0],
 [1 ,0, 0 ,1 ,1 ,1 ,0, 1 ,1, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 1 ,1 ,0, 1 ,0 ,1,0, 0, 1 ,0 ,0 ,0 ,1]]
 """

lst_s = list(range(0, n_bits))
lst_d = list(range(0, n_bits-1))
#random.shuffle(lst_s)
#random.shuffle(lst_d)
#print(lst_s)
#print(lst_d)
combos=[]
f=open("traffic"+str(rows)+"x"+str(columns)+".txt","w")
for i in lst_s:
    for j in lst_d:
        combos.append([i,j])
random.shuffle(combos)
#print(combos)
#print(len(combos))

for k in combos:
    j=k[1]
    i=k[0]
    if (j >= i):
        d = j + 1
    else:
        d = j
    if (best[i][j] >= 1):
        lbs = i // columns
        source_l = bin(lbs).replace('0b', '')
        x = source_l[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        source_l = x[::-1]
        ubs = i % columns
        source_u = bin(ubs).replace('0b', '')
        x = source_u[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        source_u = x[::-1]
        source = str(source_l) + str(source_u)
        ######################
        lbd = d // columns
        dest_l = bin(lbd).replace('0b', '')
        x = dest_l[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        dest_l = x[::-1]
        ubd = d % columns
        dest_u = bin(ubd).replace('0b', '')
        x = dest_u[::-1]  # this reverses an array
        while len(x) < 4:
            x += '0'
        dest_u = x[::-1]
        dest = str(dest_l) + str(dest_u)

        rand_ls = np.random.random_integers(0, 1, size=(13))
        rand_ls = str(rand_ls).replace(" ", "")
        rand_ls = str(rand_ls).replace("[", "")
        rand_ls = str(rand_ls).replace("]", "")
        h_f = str(source) + str(dest)
        f.write(h_f)
        f.write("\n")
        # f.write("01" + h_f)
        # f.write("\n")
        # f.write("10" + h_f)
        # f.write("\n")
        # f.write("11" + h_f)
        # f.write("\n")


f.close()


# Differential Evolution
Works completely fine, optimised

In [8]:
import random
import numpy as np
import time

# Initialize NoC parameters
rows = 16
columns = 16
n_bits = rows*columns
lst_s = list(range(0, n_bits))

# Initialize DE parameters
gen = 50
popsize = 10
crossover_rate = 0.7
mutation_rate = 0.9

# Store array of all possible paths between two nodes of the NoC
combos = np.zeros((n_bits * (n_bits-1),2))
iter = n_bits * (n_bits-1)-1
for i in lst_s:
    for j in lst_s:
        if(i != j):
            combos[iter] = [i,j]
            iter = iter - 1

# Generate the initial population
pop = np.zeros((popsize,2))
for k in range(popsize):
    i = random.choice(lst_s)
    j = random.choice(lst_s)
    pop[k] = [i,j]
  
# Define the function for which the path between two nodes is valid
def objective_function(individual):
    return(int(np.any(np.all(combos == individual, axis=1))))
 
# Define conditions and method for crossover
def crossover(pop, mutant_pop):
    crossover_pop = np.zeros((len(pop),2))
    for x in range(len(pop)):
        if random.random() < crossover_rate:
            crossover_pop[x] = mutant_pop[x]
        else:
            crossover_pop[x] = pop[x]
    return(crossover_pop)
    
# Define conditions and method for mutation
def mutant(pop):
    mutant_pop = np.zeros((len(pop),2))
    for x in range(len(pop)):
        idxs = [idx for idx in range(len(pop))]
        a, b, c = pop[random.choice(idxs)],pop[random.choice(idxs)],pop[random.choice(idxs)]
        mutant = np.add(a, mutation_rate * np.subtract(b, c))
        mutant = np.clip(mutant, 0, n_bits-1)
        mutant = np.round(mutant)
        mutant_pop[x] = mutant
    return(mutant_pop)

def differential_evolution(pop):
    unique_paths_needed = n_bits * (n_bits - 1)
    # DE has to be performed repeatedly for each generation
    for x in range(gen):
        print('Gen:',x+1)
        
        # Calculate the mutated population
        mutant_pop = mutant(pop)
        
        # Calculate the crossover population
        crossover_pop = crossover(pop, mutant_pop)
        
        pop_arr = np.array(pop)
       
        # Find the next generation by merging the current population and crossover population 
        next_pop = np.concatenate((pop_arr, crossover_pop))
        next_pop = np.unique(next_pop, axis=0)
       
        # Removing low fitness in next generation
        next_pop_fitness = np.array([objective_function(individual) for individual in next_pop])
        next_pop = np.multiply(next_pop, next_pop_fitness[:, np.newaxis])
        np.random.shuffle(next_pop)
        next_pop = next_pop.tolist()
        next_pop = [t for t in next_pop if t != [0, 0]]
        print('Population:',len(next_pop))
        pop = next_pop
        if len(pop) >= unique_paths_needed:
            print("Saturation reached at generation:", x+1)
            break

    return(pop)

start_time = time.time()
sol = differential_evolution(pop)
print('Time taken:', time.time()-start_time)

#Converting the paths into binary traffic for the testing
f=open("DE/traffic"+str(rows)+"x"+str(columns)+".txt","w")

for x in sol:
    i = x[0]
    j = x[1]
    lbs = i // columns
    source_l = bin(int(lbs)).replace('0b', '')
    x = source_l[::-1]  # this reverses an array
    while len(x) < 4:
        x += '0'
    source_l = x[::-1]
    ubs = i % columns
    source_u = bin(int(ubs)).replace('0b', '')
    x = source_u[::-1]  # this reverses an array
    while len(x) < 4:
        x += '0'
    source_u = x[::-1]
    source = str(source_l) + str(source_u)
    ######################
    lbd = j // columns
    dest_l = bin(int(lbd)).replace('0b', '')
    x = dest_l[::-1]  # this reverses an array
    while len(x) < 4:
        x += '0'
    dest_l = x[::-1]
    ubd = j % columns
    dest_u = bin(int(ubd)).replace('0b', '')
    x = dest_u[::-1]  # this reverses an array
    while len(x) < 4:
        x += '0'
    dest_u = x[::-1]
    dest = str(dest_l) + str(dest_u)    

    h_f = str(source) + str(dest)
    f.write(h_f)
    f.write("\n")


f.close()       

Gen: 1
Population: 16
Gen: 2
Population: 28
Gen: 3
Population: 43
Gen: 4
Population: 71
Gen: 5
Population: 103
Gen: 6
Population: 164
Gen: 7
Population: 261
Gen: 8
Population: 400
Gen: 9
Population: 607
Gen: 10
Population: 903
Gen: 11
Population: 1286
Gen: 12
Population: 1788
Gen: 13
Population: 2377
Gen: 14
Population: 3150
Gen: 15
Population: 4058
Gen: 16
Population: 5232
Gen: 17
Population: 6770
Gen: 18
Population: 8782
Gen: 19
Population: 11291
Gen: 20
Population: 14413
Gen: 21
Population: 18108
Gen: 22
Population: 22524
Gen: 23
Population: 27412
Gen: 24
Population: 32491
Gen: 25
Population: 37750
Gen: 26
Population: 42699
Gen: 27
Population: 47187
Gen: 28
Population: 51029
Gen: 29
Population: 54225
Gen: 30
Population: 56776
Gen: 31
Population: 58770
Gen: 32
Population: 60346
Gen: 33
Population: 61536
Gen: 34
Population: 62447
Gen: 35
Population: 63125
Gen: 36
Population: 63591
Gen: 37
Population: 64003
Gen: 38
Population: 64305
Gen: 39
Population: 64558
Gen: 40
Population: 64736
G